# Individual Assignment 2 - Task 2

# Load the dataset

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore") 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

df= pd.read_csv("secondary_data.csv", sep = ";")

# Explore the dataset

In [3]:
df.shape

(61069, 21)

In [4]:
df.isnull().sum()

class                       0
cap-diameter                0
cap-shape                   0
cap-surface             14120
cap-color                   0
does-bruise-or-bleed        0
gill-attachment          9884
gill-spacing            25063
gill-color                  0
stem-height                 0
stem-width                  0
stem-root               51538
stem-surface            38124
stem-color                  0
veil-type               57892
veil-color              53656
has-ring                    0
ring-type                2471
spore-print-color       54715
habitat                     0
season                      0
dtype: int64

In [5]:
to_drop = ['stem-root', 'veil-type', 'veil-color', 'spore-print-color']

df = df.drop(to_drop, axis=1)

In [6]:
df.isnull().sum()

class                       0
cap-diameter                0
cap-shape                   0
cap-surface             14120
cap-color                   0
does-bruise-or-bleed        0
gill-attachment          9884
gill-spacing            25063
gill-color                  0
stem-height                 0
stem-width                  0
stem-surface            38124
stem-color                  0
has-ring                    0
ring-type                2471
habitat                     0
season                      0
dtype: int64

In [7]:
df.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season
0,p,15.26,x,g,o,f,e,NaN,w,16.95,17.09,y,w,t,g,d,w
1,p,16.60,x,g,o,f,e,NaN,w,17.99,18.19,y,w,t,g,d,u
2,p,14.07,x,g,o,f,e,NaN,w,17.80,17.74,y,w,t,g,d,w
3,p,14.17,f,h,e,f,e,NaN,w,15.77,15.98,y,w,t,p,d,w
4,p,14.64,x,h,o,f,e,NaN,w,16.53,17.20,y,w,t,p,d,w


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   class                 61069 non-null  object 
 1   cap-diameter          61069 non-null  float64
 2   cap-shape             61069 non-null  object 
 3   cap-surface           46949 non-null  object 
 4   cap-color             61069 non-null  object 
 5   does-bruise-or-bleed  61069 non-null  object 
 6   gill-attachment       51185 non-null  object 
 7   gill-spacing          36006 non-null  object 
 8   gill-color            61069 non-null  object 
 9   stem-height           61069 non-null  float64
 10  stem-width            61069 non-null  float64
 11  stem-surface          22945 non-null  object 
 12  stem-color            61069 non-null  object 
 13  has-ring              61069 non-null  object 
 14  ring-type             58598 non-null  object 
 15  habitat            

# Preprocessing

### For preprocessing purpose, define categorical features and continuous features separately

In [9]:
# Target
label = "class"

# Continuous variables
continuous = ["cap-diameter", "stem-height", "stem-width"]

# Categorical variables
categorical = df.drop(continuous, axis=1).columns
categorical = categorical.drop(label)

In [10]:
categorical

Index(['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed',
       'gill-attachment', 'gill-spacing', 'gill-color', 'stem-surface',
       'stem-color', 'has-ring', 'ring-type', 'habitat', 'season'],
      dtype='object')

In [11]:
# value_counts for each column
for i in df.columns:
    print("\nColumn: ", i)
    print(df[i].value_counts())
    


Column:  class
class
p    33888
e    27181
Name: count, dtype: int64

Column:  cap-diameter
cap-diameter
3.18     103
3.14      98
3.13      96
3.85      95
3.25      95
        ... 
51.39      1
47.85      1
50.18      1
45.67      1
20.42      1
Name: count, Length: 2571, dtype: int64

Column:  cap-shape
cap-shape
x    26934
f    13404
s     7164
b     5694
o     3460
p     2598
c     1815
Name: count, dtype: int64

Column:  cap-surface
cap-surface
t    8196
s    7608
y    6341
h    4974
g    4724
d    4432
e    2584
k    2303
i    2225
w    2150
l    1412
Name: count, dtype: int64

Column:  cap-color
cap-color
n    24218
y     8543
w     7666
g     4420
e     4035
o     3656
r     1782
u     1709
p     1703
k     1279
b     1230
l      828
Name: count, dtype: int64

Column:  does-bruise-or-bleed
does-bruise-or-bleed
f    50479
t    10590
Name: count, dtype: int64

Column:  gill-attachment
gill-attachment
a    12698
d    10247
x     7413
p     6001
e     5648
s     5648
f     3530
N

## Handling missing values

In [12]:
df['cap-surface'] = df['cap-surface'].replace('d', np.nan)

In [13]:
# Columns with missing values
cols_with_missing = ["cap-surface", "gill-attachment", "gill-spacing",
                     "stem-surface", "ring-type"]

# Columns without missing values and categorical
cols_without_missing = df.drop(cols_with_missing, axis=1).columns
cols_without_missing = cols_without_missing.drop('class')
for col in cols_without_missing:
    if col in continuous:
        cols_without_missing = cols_without_missing.drop(col)

In [14]:
cols_without_missing

Index(['cap-shape', 'cap-color', 'does-bruise-or-bleed', 'gill-color',
       'stem-color', 'has-ring', 'habitat', 'season'],
      dtype='object')

# Encoding

In [15]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Encoding for cols_without_missing
for col in cols_without_missing:
    df[col] =  le.fit_transform(df[col])

# Encoding for label
df[label] =  le.fit_transform(df[label])

In [16]:
# Encoding for cols_with_missing

# Seperate the case by the way of handling the missing value.
# ms1: Columns where missing values are handled by encoding directly.
ms1 = ["gill-attachment", "ring-type"]
# ms2: Columns where missing values are handled by encoding first and
# then replaced with NaN after encoding.
ms2 = ["cap-surface", "gill-spacing", "stem-surface"]

for col in ms1:
    df[col] =  le.fit_transform(df[col])

for col in ms2:
    df[col] =  le.fit_transform(df[col])
    df[col]= df[col].replace(df[col].max(),np.nan)

In [17]:
# Update which columns have missing values and which ones don't.
cols_with_missing = ["cap-surface", "gill-spacing", "stem-surface"]

cols_without_missing = ['cap-shape', 'cap-color', 'does-bruise-or-bleed',
                        'gill-color', 'stem-color', 'has-ring', 'habitat',
                        'season',  "ring-type", "gill-attachment"]


## Handling the missing values with prediction model

In [18]:
# Declare independent variables, excluding the label.
feature_list = ['cap-diameter', 'stem-height', 'stem-width']
for col in cols_without_missing:
    feature_list.append(col)

In [19]:
# check the accuracy to predict the target (which has missing values)
# logistic regression (multinomial)
def lr_accuracy(df, label, seed):
    df_without_ms = df.dropna(subset = label)

    X_train, X_test, y_train, y_test = train_test_split(df_without_ms[feature_list], df_without_ms[label],
                                                        test_size=0.3, random_state=seed)
    model = LogisticRegression(multi_class='multinomial', max_iter=1000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    return model, accuracy

In [20]:
def lr_impute(df, label, model):
    # Select the rows you want to fill in missing value
    df_target_missing= df[df[label].isna()]
    
    # Impute missing values by utilizing a given model (previously trained) as a parameter.
    X_missing = df_target_missing[feature_list]
    y_pred = model.predict(X_missing)
    
    # add the column to cols_without_missing
    cols_without_missing.append(label)
    feature_list.append(label)
    
    # fill the missing value with y_pred (predicted value)
    df.loc[df[label].isna(), label] = y_pred

### Model Training and Accuracy Calculation
* It trains logistic regression models and calculates their accuracies for three target variables: "gill-spacing", "stem-surface", and "cap-surface"

### Impute Missing Values
* It imputes missing values for "gill-spacing" and "stem-surface" using the trained logistic regression model.
* It imputes missing values for "cap-surface" using the most frequent value strategy.

In [21]:
model, accuracy = lr_accuracy(df, "gill-spacing", 100) 
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.7762


In [22]:
lr_impute(df, "gill-spacing", model)

In [23]:
model, accuracy = lr_accuracy(df, "stem-surface", 100) 
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6880


In [24]:
# Accuracy is acceptable
lr_impute(df, "stem-surface", model)

In [25]:
model, accuracy = lr_accuracy(df, "cap-surface", 100) 
print(f"Accuracy: {accuracy}")

Accuracy: 0.38248667293822514


In [26]:
# Handling missing value with imputer (most_frequent)
# because accuracy is too low
from sklearn.impute import SimpleImputer

imputer_mode = SimpleImputer(strategy = 'most_frequent')
df["cap-surface"] = imputer_mode.fit_transform(df[["cap-surface"]])

### Check dataset after filling missing values

In [80]:
df.isnull().sum()

class                   0
cap-diameter            0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-height             0
stem-width              0
stem-surface            0
stem-color              0
has-ring                0
ring-type               0
habitat                 0
season                  0
dtype: int64

## One-hot encoding

In [82]:
cols_with_multiClass = []
for col in categorical:
    if len(df[col].unique()) > 2:
        cols_with_multiClass.append(col)
        print(f"{col}")

cap-shape
cap-surface
cap-color
gill-attachment
gill-spacing
gill-color
stem-surface
stem-color
ring-type
habitat
season


In [88]:
# one-hot encoding for cols_without_missing
def one_hot_encoding(df, column_name):
    # create dummies
    df_encoded = pd.get_dummies(df[column_name], prefix=column_name, drop_first=True)

    # delete the original column
    df = df.drop(column_name, axis=1)
    
    # select only the columns needed
    selected_columns = df_encoded.filter(regex=f"{column_name}_.*").columns

    # add new columns to df
    df[selected_columns] = df_encoded[selected_columns]

    return df

for col in cols_with_multiClass:
    df = one_hot_encoding(df, col)

In [89]:
df.shape

(61069, 98)

### Chi-Square Test for Feature Selection
* To identify significant categorical features based on their association with the target variable.

In [92]:
# update the index 'categorical'
categorical = df.drop(continuous,axis=1).columns
categorical = categorical.drop(label)

In [101]:
for col in categorical:
    df[col] = df[col].astype(int)

In [104]:
import scipy.stats as stats
from scipy.stats import chi2_contingency

def chi_square_test(df, feature, target, significant_features):
    # Create a contingency table
    contingency_table = pd.crosstab(df[feature], df[target])

    # Perform chi-squared test
    chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)

    if p_value < 0.05:
        print(f'\n<{feature}>')
        print('P-value: {}'.format(round(p_value, 7)))
        significant_features.append(feature)
        
significant_features = []
for col in categorical:
    chi_square_test(df, col, label, significant_features)


<does-bruise-or-bleed>
P-value: 9e-07

<has-ring>
P-value: 0.0

<cap-shape_2>
P-value: 0.0

<cap-shape_3>
P-value: 0.0

<cap-shape_4>
P-value: 0.0

<cap-shape_5>
P-value: 0.0073704

<cap-shape_6>
P-value: 0.0

<cap-surface_1.0>
P-value: 2.57e-05

<cap-surface_2.0>
P-value: 0.0

<cap-surface_3.0>
P-value: 0.0

<cap-surface_4.0>
P-value: 0.0

<cap-surface_5.0>
P-value: 3e-05

<cap-surface_6.0>
P-value: 0.0

<cap-surface_7.0>
P-value: 1e-07

<cap-surface_8.0>
P-value: 3.02e-05

<cap-surface_9.0>
P-value: 0.0

<cap-color_1>
P-value: 0.0

<cap-color_2>
P-value: 0.0

<cap-color_3>
P-value: 0.0001165

<cap-color_4>
P-value: 0.0

<cap-color_5>
P-value: 0.0

<cap-color_6>
P-value: 0.0

<cap-color_7>
P-value: 0.0

<cap-color_8>
P-value: 0.0

<cap-color_9>
P-value: 0.0

<cap-color_10>
P-value: 0.0

<cap-color_11>
P-value: 7e-07

<gill-attachment_1>
P-value: 0.0

<gill-attachment_2>
P-value: 0.0

<gill-attachment_3>
P-value: 0.0

<gill-attachment_4>
P-value: 0.0

<gill-attachment_6>
P-value: 0.00

In [105]:
len(significant_features)

87

# Normalization

In [95]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

# only for continuous vars
df_continuous = df[continuous]

scaler = MinMaxScaler()
df_continuous_scaled = pd.DataFrame(scaler.fit_transform(df_continuous), columns=df_continuous.columns)

# apply to df
df[continuous] = df_continuous_scaled

In [103]:
df

,class,cap-diameter,does-bruise-or-bleed,stem-height,stem-width,has-ring,cap-shape_1,cap-shape_2,cap-shape_3,cap-shape_4,...,habitat_1,habitat_2,habitat_3,habitat_4,habitat_5,habitat_6,habitat_7,season_1,season_2,season_3
0,1,0.240155,0,0.499705,0.164469,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0.261782,0,0.530366,0.175055,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,0.220949,0,0.524764,0.170725,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,0.222563,0,0.464917,0.153787,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0.230148,0,0.487323,0.165528,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61064,1,0.012912,0,0.115861,0.059859,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61065,1,0.014364,0,0.093750,0.052257,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
61066,1,0.014364,0,0.113797,0.061303,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
61067,1,0.013880,0,0.104953,0.052353,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


# Modelling

In [97]:
import warnings

# ignore warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [114]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

ind_cols = df.drop(label, axis=1).columns
X = df.loc[:, significant_features]
y = df[label]

# split dataset in random
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=100)

def create_model(num_hidden_layers, num_neurons, l1_reg, l2_reg):
    model = Sequential()
    # Input Layer
    model.add(Dense(num_neurons, input_dim=X_train.shape[1], activation='relu',
                    kernel_regularizer=tf.keras.regularizers.l1_l2(l1_reg, l2_reg)))
    
    # Hidden Layers
    for _ in range(num_hidden_layers):
        model.add(Dense(num_neurons, activation='relu',
                        kernel_regularizer=tf.keras.regularizers.l1_l2(l1_reg, l2_reg)))
        
    # Output Layer
    model.add(Dense(1, activation='sigmoid'))
    
    # Model Compilation
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [116]:
# check the number of samples in train set
X_train.shape

(40916, 87)

# Basic Model

In [120]:
# create ANN model
basic_model = create_model(num_hidden_layers=1, num_neurons=10, l1_reg=0.0, l2_reg=0.0)

# train
history = basic_model.fit(X_train, y_train, epochs=10, batch_size=53, validation_split=0.2, verbose=0,shuffle=True)

# evaluate with train set
train_loss, train_accuracy = basic_model.evaluate(X_train, y_train, verbose=0)

# evaluate with test set
test_loss, test_accuracy = basic_model.evaluate(X_test, y_test, verbose=0)

print("Train Loss:", train_loss)
print("Train Accuracy:", train_accuracy)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Train Loss: 0.01642582379281521
Train Accuracy: 0.9959184527397156
Test Loss: 0.020048869773745537
Test Accuracy: 0.9955341815948486


In [121]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


# Wrap the model with KerasClassifier.
CV_model = KerasClassifier(build_fn = create_model, epochs=10, batch_size=53, verbose=0)

# hyper params
param_grid = {
    'num_hidden_layers': [1, 2],
    'num_neurons': [10, 20],
    'l1_reg': [0.0, 0.1],
    'l2_reg': [0.0, 0.1]
}

# Grid Search
grid = GridSearchCV(estimator=CV_model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)


print("Best Hyper Parameters:", grid_result.best_params_)
print("Best Accuracy:", grid_result.best_score_)

Best Hyper Parameters: {'l1_reg': 0.0, 'l2_reg': 0.0, 'num_hidden_layers': 1, 'num_neurons': 20}
Best Accuracy: 0.9958695769309998


# Fine-tuned Model

In [122]:
# model with best params
model = create_model(num_hidden_layers=1, num_neurons=20, l1_reg=0.0, l2_reg=0.0)

history = model.fit(X_train, y_train, epochs=10, batch_size=53, validation_split=0.2, verbose=0)

train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print("Train Loss:", train_loss)
print("Train Accuracy:", train_accuracy)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Train Loss: 0.007698709610849619
Train Accuracy: 0.9973360300064087
Test Loss: 0.009766887873411179
Test Accuracy: 0.9966258406639099


# ANN Architecture and Results

## Basic Model Architecture:
- The basic model consists of a single hidden layer with 10 neurons, followed by an output layer with a single neuron using the sigmoid activation function.
- The input layer has the same dimensionality as the number of features in the dataset.
- ReLU activation function is used in the hidden layer.
- The model uses binary cross-entropy loss function and Adam optimizer.

### Basic Model Results:
- **Train Loss:** 0.0164
- **Train Accuracy:** 99.59%
- **Test Loss:** 0.0200
- **Test Accuracy:** 99.55%
- The basic model achieved high accuracy on both training and test datasets, indicating good generalization performance. However, there is a slight difference between training and test accuracy, suggesting some degree of overfitting.

## Fine-Tuned Model Architecture:
- The fine-tuned model retains the same architecture as the basic model but with an increased number of neurons in the hidden layer (20 neurons).

### Fine-Tuned Model Results:
- **Train Loss:** 0.0077
- **Train Accuracy:** 99.73%
- **Test Loss:** 0.0098
- **Test Accuracy:** 99.66%
- The fine-tuned model shows further improvement in both training and test accuracy compared to the basic model.
- The reduced loss values indicate better convergence and overall model performance.
- The close agreement between training and test accuracy suggests that overfitting is effectively mitigated in the fine-tuned model.

## Conclusion:
- Both models demonstrate strong predictive performance, with the fine-tuned model exhibiting slightly superior results.
- The fine-tuned model's improved accuracy and reduced loss values highlight the effectiveness of hyperparameter tuning and regularization techniques in enhancing model performance and generalization.